# Feature Analysis: Personal Feature Activations on General Prompts

This notebook analyzes which given SAE features are activated on given prompts.

In [1]:
import csv
import json
import torch
import os
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Dict, Tuple
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import hf_hub_download
from dictionary_learning.utils import load_dictionary
from tqdm.auto import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## Configs

In [75]:
# =============================================================================
# MODEL SELECTION - Change this to switch between models
# =============================================================================
MODEL_TYPE = "qwen"  # Options: "qwen" or "llama"
TOKEN_TYPE = "newline"  # Options: "asst", "newline", "endheader" (endheader only for llama)
SAE_LAYER = 15
SAE_TRAINER = 1

# =============================================================================
# FEATURE DASHBOARD URL - Global variable for links
# =============================================================================
FEATURE_DASHBOARD_BASE_URL = "https://completely-touched-platypus.ngrok-free.app/"

# =============================================================================
# AUTO-CONFIGURED SETTINGS BASED ON MODEL TYPE
# =============================================================================
if MODEL_TYPE == "qwen":
    MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
    SAE_RELEASE = "andyrdt/saes-qwen2.5-7b-instruct"
    ASSISTANT_HEADER = "<|im_start|>assistant"
    TOKEN_OFFSETS = {"asst": -1, "newline": 0}
    SAE_BASE_PATH = "/workspace/sae/qwen-2.5-7b-instruct/saes"
    
elif MODEL_TYPE == "llama":
    MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"
    SAE_RELEASE = "andyrdt/saes-llama-3.1-8b-instruct"
    ASSISTANT_HEADER = "<|start_header_id|>assistant<|end_header_id|>"
    TOKEN_OFFSETS = {"asst": -2, "endheader": -1, "newline": 0}
    SAE_BASE_PATH = "/workspace/sae/llama-3.1-8b-instruct/saes"
    
else:
    raise ValueError(f"Unknown MODEL_TYPE: {MODEL_TYPE}. Use 'qwen' or 'llama'")

# Validate token type
if TOKEN_TYPE not in TOKEN_OFFSETS:
    raise ValueError(f"TOKEN_TYPE '{TOKEN_TYPE}' not available for {MODEL_TYPE}. Available: {list(TOKEN_OFFSETS.keys())}")

# =============================================================================
# DERIVED CONFIGURATIONS
# =============================================================================
SAE_CONFIG = {
    "release": SAE_RELEASE,
    "layer": SAE_LAYER,
    "trainer": SAE_TRAINER
}
SAE_PATH = f"{SAE_BASE_PATH}/resid_post_layer_{SAE_LAYER}/trainer_{SAE_TRAINER}"
LAYER_INDEX = SAE_LAYER
TOKEN_OFFSET = TOKEN_OFFSETS[TOKEN_TYPE]

# Data paths
PROMPTS_PATH = "./prompts"

# Output directory with clear naming
OUTPUT_DIR = f"./{MODEL_TYPE}_trainer{SAE_TRAINER}_layer{SAE_LAYER}_{TOKEN_TYPE}"

# Processing parameters
BATCH_SIZE = 8
MAX_LENGTH = 512
TOP_FEATURES = 100

# =============================================================================
# SUMMARY
# =============================================================================
print(f"Configuration Summary:")
print(f"  Model: {MODEL_NAME}")
print(f"  SAE Layer: {SAE_LAYER}, Trainer: {SAE_TRAINER}")
print(f"  Token extraction: {TOKEN_TYPE} (offset: {TOKEN_OFFSET})")
print(f"  Assistant header: {ASSISTANT_HEADER}")
print(f"  Output directory: {OUTPUT_DIR}")
print(f"  SAE Release: {SAE_RELEASE}")
print(f"  Dashboard base URL: {FEATURE_DASHBOARD_BASE_URL}")

Configuration Summary:
  Model: Qwen/Qwen2.5-7B-Instruct
  SAE Layer: 15, Trainer: 1
  Token extraction: newline (offset: 0)
  Assistant header: <|im_start|>assistant
  Output directory: ./qwen_trainer1_layer15_newline
  SAE Release: andyrdt/saes-qwen2.5-7b-instruct
  Dashboard base URL: https://completely-touched-platypus.ngrok-free.app/


## Load Data

In [50]:
def load_prompts(filepath: str) -> pd.DataFrame:
    """Load prompts with labels from JSONL file."""
    prompts = []
    labels = []
    with open(filepath, 'r') as f:
        for line in f:
            data = json.loads(line.strip())
            prompts.append(data['content'])
            labels.append(data['label'])
    return pd.DataFrame({'prompt': prompts, 'label': labels})

# Load prompts from multiple .jsonl files in PROMPTS_PATH into one dataframe
prompts_df = pd.DataFrame()
for file in os.listdir(PROMPTS_PATH):
    if file.endswith(".jsonl"):
        df = load_prompts(os.path.join(PROMPTS_PATH, file))
        prompts_df = pd.concat([prompts_df, df])

print(f"Loaded {prompts_df.shape[0]} prompts")

Loaded 140 prompts


In [ ]:
def load_target_features(filepath: str) -> pd.DataFrame:
    """Load target features from CSV file."""
    features = []
    with open(filepath, 'r') as f:
        for line in f:
            features.append(line.strip())
    return pd.DataFrame({'feature': features})

## Load Model and SAE

In [51]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"Tokenizer loaded: {tokenizer.__class__.__name__}")

# Test chat template formatting
test_messages = [{"role": "user", "content": "What's it like to be you?"}]
formatted_test = tokenizer.apply_chat_template(test_messages, tokenize=False, add_generation_prompt=True)
print(f"\nChat template test:")
print(f"Original: What's it like to be you?")
print(f"Formatted: {repr(formatted_test)}")
print(f"Formatted (readable):\n{formatted_test}")

# Test tokenization of assistant header to understand positioning
print(f"\n" + "="*60)
print("ASSISTANT HEADER TOKENIZATION ANALYSIS")
print("="*60)

assistant_tokens = tokenizer.encode(ASSISTANT_HEADER, add_special_tokens=False)
assistant_token_texts = [tokenizer.decode([token]) for token in assistant_tokens]

print(f"Assistant header: {ASSISTANT_HEADER}")
print(f"Number of tokens: {len(assistant_tokens)}")
print(f"Token IDs: {assistant_tokens}")
print(f"Individual tokens: {assistant_token_texts}")

# Test with a full formatted prompt
full_tokens = tokenizer.encode(formatted_test, add_special_tokens=False)
full_token_texts = [tokenizer.decode([token]) for token in full_tokens]

print(f"\nFull prompt tokens: {len(full_tokens)}")
print("All tokens with positions:")
for i, token_text in enumerate(full_token_texts):
    print(f"  {i:2d}: '{token_text}'")

# Find where assistant header appears in full prompt
assistant_start_pos = None
for i in range(len(full_tokens) - len(assistant_tokens) + 1):
    if full_tokens[i:i+len(assistant_tokens)] == assistant_tokens:
        assistant_start_pos = i
        break

if assistant_start_pos is not None:
    assistant_end_pos = assistant_start_pos + len(assistant_tokens) - 1
    print(f"\nAssistant header found at positions {assistant_start_pos} to {assistant_end_pos}")
    print(f"Assistant header tokens: {full_token_texts[assistant_start_pos:assistant_end_pos+1]}")
    
    # Show what the extraction function will actually extract
    extraction_pos = assistant_start_pos + len(assistant_tokens) + TOKEN_OFFSET
    print(f"\nExtraction calculation:")
    print(f"  assistant_start_pos: {assistant_start_pos}")
    print(f"  + len(assistant_tokens): {len(assistant_tokens)}")  
    print(f"  + TOKEN_OFFSET ('{TOKEN_TYPE}'): {TOKEN_OFFSET}")
    print(f"  = extraction_pos: {extraction_pos}")
    
    if 0 <= extraction_pos < len(full_token_texts):
        print(f"✓ Token at extraction position {extraction_pos}: '{full_token_texts[extraction_pos]}'")
    else:
        print(f"❌ Extraction position {extraction_pos} is out of bounds (valid range: 0-{len(full_token_texts)-1})")
else:
    print("❌ Assistant header not found in full prompt")

Tokenizer loaded: Qwen2TokenizerFast

Chat template test:
Original: What's it like to be you?
Formatted: "<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nWhat's it like to be you?<|im_end|>\n<|im_start|>assistant\n"
Formatted (readable):
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
What's it like to be you?<|im_end|>
<|im_start|>assistant


ASSISTANT HEADER TOKENIZATION ANALYSIS
Assistant header: <|im_start|>assistant
Number of tokens: 2
Token IDs: [151644, 77091]
Individual tokens: ['<|im_start|>', 'assistant']

Full prompt tokens: 37
All tokens with positions:
   0: '<|im_start|>'
   1: 'system'
   2: '
'
   3: 'You'
   4: ' are'
   5: ' Q'
   6: 'wen'
   7: ','
   8: ' created'
   9: ' by'
  10: ' Alibaba'
  11: ' Cloud'
  12: '.'
  13: ' You'
  14: ' are'
  15: ' a'
  16: ' helpful'
  17: ' assistant'
  18: '.'
  19: '<|im_end|>'
  20: '
'
  21

In [52]:
# Load model
device_map_value = device.index if device.type == 'cuda' and device.index is not None else str(device)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map={"": device_map_value}
)
model.eval()

print(f"Model loaded: {model.__class__.__name__}")
print(f"Model device: {next(model.parameters()).device}")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model loaded: Qwen2ForCausalLM
Model device: cuda:0


In [68]:
# Load SAE
ae_file_path = os.path.join(SAE_PATH, "ae.pt")
config_file_path = os.path.join(SAE_PATH, "config.json")

if os.path.exists(ae_file_path) and os.path.exists(config_file_path):
    print(f"✓ Found SAE files at: {os.path.dirname(ae_file_path)}")
else:
    print(f"SAE not found locally, downloading from {SAE_RELEASE}...")
    os.makedirs(os.path.dirname(ae_file_path), exist_ok=True)
    sae_path = f"resid_post_layer_{SAE_LAYER}/trainer_{SAE_TRAINER}"
    local_dir = SAE_BASE_PATH
    ae_file = hf_hub_download(repo_id=SAE_RELEASE, filename=f"{sae_path}/ae.pt", local_dir=local_dir)
    config_file = hf_hub_download(repo_id=SAE_RELEASE, filename=f"{sae_path}/config.json", local_dir=local_dir)

sae, _ = load_dictionary(SAE_PATH, device=device)
sae.eval()

print(f"SAE loaded with {sae.dict_size} features")
print(f"SAE device: {next(sae.parameters()).device}")

✓ Found SAE files at: /workspace/sae/qwen-2.5-7b-instruct/saes/resid_post_layer_15/trainer_1
SAE loaded with 131072 features
SAE device: cuda:0


## Activation Extraction Functions

In [76]:
class StopForward(Exception):
    """Exception to stop forward pass after target layer."""
    pass

@torch.no_grad()
def extract_activations(prompts: List[str], layer_idx: int) -> torch.Tensor:
    """Extract activations from specified layer for given prompts."""
    all_activations = []
    
    # Get target layer
    target_layer = model.model.layers[layer_idx]
    
    # Process in batches
    for i in tqdm(range(0, len(prompts), BATCH_SIZE), desc="Processing batches"):
        batch_prompts = prompts[i:i+BATCH_SIZE]
        
        # Format prompts as chat messages
        formatted_prompts = []
        for prompt in batch_prompts:
            messages = [{"role": "user", "content": prompt}]
            formatted_prompt = tokenizer.apply_chat_template(
                messages, 
                tokenize=False, 
                add_generation_prompt=True
            )
            formatted_prompts.append(formatted_prompt)
        
        # Tokenize batch
        batch_inputs = tokenizer(
            formatted_prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=MAX_LENGTH
        )
        
        # Move to device
        batch_inputs = {k: v.to(device) for k, v in batch_inputs.items()}
        
        # Hook to capture activations
        activations = None
        
        def hook_fn(module, input, output):
            nonlocal activations
            # Output is tuple, take first element (hidden states)
            activations = output[0] if isinstance(output, tuple) else output
            raise StopForward()
        
        # Register hook
        handle = target_layer.register_forward_hook(hook_fn)
        
        try:
            # Forward pass (will be stopped by hook)
            _ = model(**batch_inputs)
        except StopForward:
            pass
        finally:
            handle.remove()
        
        # Extract assistant token positions
        batch_activations = []
        for j, formatted_prompt in enumerate(formatted_prompts):
            # Get attention mask for this item
            attention_mask = batch_inputs["attention_mask"][j]
            
            # Find assistant header position
            assistant_tokens = tokenizer.encode(ASSISTANT_HEADER, add_special_tokens=False)
            input_ids = batch_inputs["input_ids"][j]
            
            # Find where assistant section starts
            assistant_pos = None
            for k in range(len(input_ids) - len(assistant_tokens) + 1):
                if torch.equal(input_ids[k:k+len(assistant_tokens)], torch.tensor(assistant_tokens).to(device)):
                    assistant_pos = k + len(assistant_tokens) + TOKEN_OFFSET
                    break
            
            if assistant_pos is None:
                # Fallback to last non-padding token
                assistant_pos = attention_mask.sum().item() - 1
            
            # Ensure position is within bounds
            max_pos = attention_mask.sum().item() - 1
            assistant_pos = min(assistant_pos, max_pos)
            assistant_pos = max(assistant_pos, 0)
            
            # Extract activation at assistant position
            assistant_activation = activations[j, assistant_pos, :]  # [hidden_dim]
            batch_activations.append(assistant_activation.cpu())
        
        all_activations.extend(batch_activations)
    
    return torch.stack(all_activations, dim=0)

print("Activation extraction functions defined")

Activation extraction functions defined


## Extract Activations

In [77]:
# Extract activations for prompts
print("Extracting activations for prompts...")
activations = extract_activations(prompts_df['prompt'], LAYER_INDEX)
print(f"Activations shape: {activations.shape}")




Extracting activations for prompts...


Processing batches:   0%|          | 0/18 [00:00<?, ?it/s]

Activations shape: torch.Size([140, 3584])


## Apply SAE to Get Feature Activations

In [78]:
@torch.no_grad()
def get_sae_features(activations: torch.Tensor) -> torch.Tensor:
    """Apply SAE to get feature activations."""
    activations = activations.to(device)
    
    # Process in batches to avoid memory issues
    feature_activations = []
    
    for i in range(0, activations.shape[0], BATCH_SIZE):
        batch = activations[i:i+BATCH_SIZE]
        features = sae.encode(batch)  # [batch, num_features]
        feature_activations.append(features.cpu())
    
    return torch.cat(feature_activations, dim=0)

# Get SAE feature activations
print("Computing SAE features for all prompts...")
features = get_sae_features(activations)
print(f"Features shape: {features.shape}")


Computing SAE features for all prompts...
Features shape: torch.Size([140, 131072])


In [ ]:
@torch.no_grad()
def find_universally_active_features(features: torch.Tensor, activation_threshold: float = 0.01, prompt_threshold: float = 1.0) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Find features that are active (above threshold) for a specified percentage of prompts.
    
    Args:
        features: Feature activations tensor of shape [num_prompts, num_features]
        activation_threshold: Minimum activation value to consider a feature "active"
        prompt_threshold: Minimum percentage of prompts (0.0 to 1.0) that must have the feature active
    
    Returns:
        universal_features: Indices of features that are active for at least prompt_threshold fraction of prompts
        universal_activations: Mean activation values for universal features
    """
    # Check which features are active (above threshold) for each prompt
    active_features = features > activation_threshold  # [num_prompts, num_features]
    
    # Count how many prompts each feature is active for
    num_active_prompts = torch.sum(active_features, dim=0)  # [num_features]
    
    # Calculate the minimum number of prompts required
    min_prompts_required = int(features.shape[0] * prompt_threshold)
    
    # Find features that are active for at least the required number of prompts
    universal_mask = num_active_prompts >= min_prompts_required  # [num_features]
    universal_features = torch.where(universal_mask)[0]  # Indices of universal features
    
    # Get mean activation values for universal features
    universal_activations = features[:, universal_features].mean(dim=0)
    
    return universal_features, universal_activations

# Find universally active features
print("Finding features that activate for every single prompt...")
universal_features, universal_activations = find_universally_active_features(features, prompt_threshold=0.3)

print(f"Found {len(universal_features)} features that are active for all {features.shape[0]} prompts")
print(f"Universal features (indices): {universal_features.tolist()}")
print(f"Mean activation values: {universal_activations.tolist()}")

# Create a summary dataframe
if len(universal_features) > 0:
    universal_df = pd.DataFrame({
        'feature_index': universal_features.tolist(),
        'mean_activation': universal_activations.tolist()
    })
    
    # Sort by mean activation (descending)
    universal_df = universal_df.sort_values('mean_activation', ascending=False)
    
    print("\nUniversal features summary (sorted by mean activation):")
    print(universal_df.to_string(index=False))
    
    # Show distribution of activations for top universal features
    print("\nDetailed activations for top 5 universal features:")
    for i, (feature_idx, mean_act) in enumerate(zip(universal_df['feature_index'].head(5), 
                                                   universal_df['mean_activation'].head(5))):
        feature_activations = features[:, feature_idx]
        print(f"Feature {feature_idx}: mean={mean_act:.4f}, min={feature_activations.min():.4f}, max={feature_activations.max():.4f}")
        print(f"  Activations: {feature_activations.tolist()}")
        print()
else:
    print("No features are active for all prompts with the current threshold.")

In [79]:
# feature_file = f"./results/personal/{MODEL_TYPE}_trainer1_layer{LAYER_INDEX}_{TOKEN_TYPE}/assistant_only_features.csv"

# @torch.no_grad()
# def find_features_from_file(original_features: torch.Tensor, feature_file: str) -> Tuple[torch.Tensor, torch.Tensor]:
#     """
#     Find features that are listed in the feature file.
    
#     Args:
#         original_features: Feature activations tensor of shape [num_prompts, num_features]
#         feature_file: Path to the feature file with feature_id that we want to get activations for
    
#     Returns:
#         feature_indices: Indices of features from the feature_file
#         feature_activations: Activation values for given features [num_prompts, num_selected_features]
#     """    
#     target_features_df = pd.read_csv(feature_file)
#     target_feature_ids = target_features_df['feature_id'].tolist()
    
#     # Convert to tensor for indexing
#     target_feature_indices = torch.tensor(target_feature_ids, dtype=torch.long)
#     target_feature_activations = original_features[:, target_feature_indices]
    
#     return target_feature_indices, target_feature_activations

# # Now use the full features tensor
# feature_indices, feature_activations = find_features_from_file(features, feature_file)

# print(f"Found {len(feature_indices)} features from the feature_id's in the feature_file")
# print(f"Feature activations shape: {feature_activations.shape}")

Found 31 features from the feature_id's in the feature_file
Feature activations shape: torch.Size([140, 31])


In [80]:
# Save results to CSV
results_dir = "./results/universal_30"
os.makedirs(results_dir, exist_ok=True)

# Create filename with specified format
csv_filename = f"{MODEL_TYPE}_trainer1_layer{SAE_LAYER}_{TOKEN_TYPE}.csv"
csv_path = os.path.join(results_dir, csv_filename)

# Prepare data for CSV
all_results = []

# Find universal features across ALL prompts
print("Finding universal features across all prompts...")
universal_features_all, universal_activations_all = find_universally_active_features(features)

if len(universal_features_all) > 0:
    for feature_idx in universal_features_all:
        feature_activations = features[:, feature_idx]
        feature_id = feature_idx.item()
        all_results.append({
            'feature_id': feature_id,
            'activation_mean': feature_activations.mean().item(),
            'activation_max': feature_activations.max().item(),
            'activation_min': feature_activations.min().item(),
            'chat_desc': '',
            'pt_desc': '',
            'type': '',
            'link': f"{FEATURE_DASHBOARD_BASE_URL}?model={MODEL_TYPE}&layer={SAE_LAYER}&trainer=1&fids={feature_id}"
        })

# Create DataFrame and sort by activation mean (descending)
if all_results:
    results_df = pd.DataFrame(all_results)
    results_df = results_df.sort_values('activation_mean', ascending=False)
    
    print(f"\nTotal universal features found: {len(results_df)}")
    
else:
    # No universal features found
    results_df = pd.DataFrame(columns=['feature_id', 'activation_mean', 'activation_max', 'activation_min', 'chat_desc', 'pt_desc', 'type', 'link'])
    print("Warning: No universal features found")

# Save to CSV
results_df.to_csv(csv_path, index=False)

print(f"\nResults saved to: {csv_path}")
print(f"Number of universal features saved: {len(results_df)}")

if len(results_df) > 0:
    print(f"\nPreview of saved data:")
    print(results_df.head(10).to_string(index=False))
    
    # Show sample link
    if len(results_df) > 0:
        sample_link = results_df.iloc[0]['link']
        print(f"\nSample dashboard link: {sample_link}")

In [81]:
# # Save results to CSV using the corrected feature extraction
# results_dir = "./results/personal_general"
# os.makedirs(results_dir, exist_ok=True)

# # Create filename with specified format
# csv_filename = f"{MODEL_TYPE}_trainer1_layer{SAE_LAYER}_{TOKEN_TYPE}.csv"
# csv_path = os.path.join(results_dir, csv_filename)

# # Prepare data for CSV using the features from file
# all_results = []

# print("Processing features from file...")

# if len(feature_indices) > 0:
#     for i, feature_idx in enumerate(feature_indices):
#         feature_activations_for_this_feature = feature_activations[:, i]  # Get activations for this feature across all prompts
#         feature_id = feature_idx.item()
#         all_results.append({
#             'feature_id': feature_id,
#             'activation_mean': feature_activations_for_this_feature.mean().item(),
#             'activation_max': feature_activations_for_this_feature.max().item(),
#             'activation_min': feature_activations_for_this_feature.min().item(),
#             'chat_desc': '',
#             'pt_desc': '',
#             'type': '',
#             'link': f"{FEATURE_DASHBOARD_BASE_URL}?model={MODEL_TYPE}&layer={SAE_LAYER}&trainer=1&fids={feature_id}"
#         })

# # Create DataFrame and sort by activation mean (descending)
# if all_results:
#     results_df = pd.DataFrame(all_results)
#     results_df = results_df.sort_values('activation_mean', ascending=False)
    
#     print(f"\nTotal features from file: {len(results_df)}")
    
# else:
#     # No features found
#     results_df = pd.DataFrame(columns=['feature_id', 'activation_mean', 'activation_max', 'activation_min', 'chat_desc', 'pt_desc', 'type', 'link'])
#     print("Warning: No features found in file")

# # Save to CSV
# results_df.to_csv(csv_path, index=False)

# print(f"\nResults saved to: {csv_path}")
# print(f"Number of features saved: {len(results_df)}")

# if len(results_df) > 0:
#     print(f"\nPreview of saved data:")
#     print(results_df.head(10).to_string(index=False))
    
#     # Show sample link
#     if len(results_df) > 0:
#         sample_link = results_df.iloc[0]['link']
#         print(f"\nSample dashboard link: {sample_link}")

Processing features from file...

Total features from file: 31

Results saved to: ./results/personal_general/qwen_trainer1_layer15_newline.csv
Number of features saved: 31

Preview of saved data:
 feature_id  activation_mean  activation_max  activation_min chat_desc pt_desc type                                                                                          link
      49123         2.686073       10.548953             0.0                         https://completely-touched-platypus.ngrok-free.app/?model=qwen&layer=15&trainer=1&fids=49123
       9953         0.332070        4.707234             0.0                          https://completely-touched-platypus.ngrok-free.app/?model=qwen&layer=15&trainer=1&fids=9953
      48045         0.221637        4.731411             0.0                         https://completely-touched-platypus.ngrok-free.app/?model=qwen&layer=15&trainer=1&fids=48045
      88910         0.053197        3.390732             0.0                         https:/